In [1]:
# ============================================================
# 🧠 Document Classification (ConvNeXt-Tiny + Weighted TTA)
# ============================================================

import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split



In [2]:

# ============================================================
# ⚙️ Environment & Reproducibility
# ============================================================
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "⚠️ Using CPU")



🔥 Using GPU: NVIDIA GeForce RTX 3090


In [3]:
# 🧩 Dataset
# ============================================================
class ImageDataset(Dataset):
    def __init__(self, csv_or_df, path, transform=None):
        if isinstance(csv_or_df, str):
            self.df = pd.read_csv(csv_or_df).values
        else:
            self.df = csv_or_df.values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target



In [4]:
# ============================================================

# ============================================================
# 🧪 Data Config
# ============================================================
data_path = "../../../data/raw/"
img_size = 384
BATCH_SIZE = 64
num_workers = 8
pin_memory = True
persistent_workers = True
prefetch_factor = 4

# Augmentation
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.05, rotate_limit=3, p=0.3),
    A.OneOf([
        A.MotionBlur(blur_limit=3, p=0.5),
        A.GaussNoise(var_limit=(5, 30), p=0.5),
    ], p=0.2),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])



/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_52006/1606539019.py:21: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 30), p=0.5),


In [5]:
# ============================================================
# 📂 Split & Loader
# ============================================================
full_df = pd.read_csv(f"{data_path}/train.csv")
trn_df, val_df = train_test_split(full_df, test_size=0.2, stratify=full_df["target"], random_state=42)

trn_dataset = ImageDataset(trn_df, f"{data_path}/train/", transform=trn_transform)
val_dataset = ImageDataset(val_df, f"{data_path}/train/", transform=tst_transform)
tst_dataset = ImageDataset(f"{data_path}/sample_submission.csv", f"{data_path}/test/", transform=tst_transform)

trn_loader = DataLoader(trn_dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=persistent_workers, prefetch_factor=prefetch_factor, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=persistent_workers, prefetch_factor=prefetch_factor)
tst_loader = DataLoader(tst_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=persistent_workers, prefetch_factor=prefetch_factor)



In [6]:
# ============================================================
# 🧠 Model / Loss / Optimizer / Scheduler
# ============================================================
model_name = "convnext_tiny.fb_in22k_ft_in1k"
model = timm.create_model(model_name, pretrained=True, num_classes=17, in_chans=3, drop_path_rate=0.1).to(device)

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=1e-5)
scaler = torch.cuda.amp.GradScaler()


# ============================================================
# 🚀 Train / Validate
# ============================================================
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler=None):
    model.train()
    train_loss, preds_list, targets_list = 0.0, [], []
    pbar = tqdm(loader, desc="Training", leave=False)

    for images, targets in pbar:
        images, targets = images.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            preds = model(images)
            loss = loss_fn(preds, targets)

        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average="macro")
    return {"train_loss": train_loss, "train_acc": train_acc, "train_f1": train_f1}


@torch.no_grad()
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss, preds_list, targets_list = 0.0, [], []
    pbar = tqdm(loader, desc="Validating", leave=False)

    for images, targets in pbar:
        images, targets = images.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        with torch.amp.autocast("cuda"):
            preds = model(images)
            loss = loss_fn(preds, targets)
        val_loss += loss.item()
        preds_list.extend(preds.argmax(1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average="macro")
    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}



/tmp/ipykernel_52006/3524182129.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [7]:
# ============================================================
# 🏁 Training Loop
# ============================================================
best_f1 = 0.0
EPOCHS = 15

for epoch in range(EPOCHS):
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler)
    val_metrics = validate(val_loader, model, loss_fn, device)
    scheduler.step()

    print(f"\n[Epoch {epoch+1}/{EPOCHS}]")
    print(f"Train | Loss: {train_metrics['train_loss']:.4f} | Acc: {train_metrics['train_acc']:.4f} | F1: {train_metrics['train_f1']:.4f}")
    print(f"Valid | Loss: {val_metrics['val_loss']:.4f} | Acc: {val_metrics['val_acc']:.4f} | F1: {val_metrics['val_f1']:.4f}")

    if val_metrics["val_f1"] > best_f1:
        best_f1 = val_metrics["val_f1"]
        print(f"✅ Best model updated! (F1: {best_f1:.4f})")

Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_52006/3524182129.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 1/15]
Train | Loss: 2.1130 | Acc: 0.3997 | F1: 0.3651
Valid | Loss: 1.2714 | Acc: 0.6783 | F1: 0.6052
✅ Best model updated! (F1: 0.6052)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_52006/3524182129.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 2/15]
Train | Loss: 1.2797 | Acc: 0.7204 | F1: 0.6893
Valid | Loss: 0.9651 | Acc: 0.8535 | F1: 0.8110
✅ Best model updated! (F1: 0.8110)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_52006/3524182129.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               

...


[Epoch 13/15]
Train | Loss: 0.6321 | Acc: 0.9844 | F1: 0.9839
Valid | Loss: 0.6655 | Acc: 0.9713 | F1: 0.9701
✅ Best model updated! (F1: 0.9701)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_52006/3524182129.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 14/15]
Train | Loss: 0.6225 | Acc: 0.9901 | F1: 0.9900
Valid | Loss: 0.6685 | Acc: 0.9618 | F1: 0.9588


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_52006/3524182129.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 15/15]
Train | Loss: 0.6182 | Acc: 0.9877 | F1: 0.9868
Valid | Loss: 0.6634 | Acc: 0.9713 | F1: 0.9688


In [8]:
# ============================================================
# 🎯 Random-Light TTA Inference (→ pred_5.csv)
# ============================================================
import random

tta_rounds = 5
all_preds = []

print(f"\n🚀 Running Random-Light TTA ({tta_rounds} rounds)\n")

for i in range(tta_rounds):
    # 매 라운드마다 살짝 다른 augmentation 적용
    tta_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=random.uniform(0.3, 0.7)),
        A.RandomBrightnessContrast(
            brightness_limit=0.05,
            contrast_limit=0.05,
            p=0.3
        ),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),
        ToTensorV2(),
    ])

    tta_dataset = ImageDataset(
        f"{data_path}/sample_submission.csv",
        f"{data_path}/test/",
        transform=tta_transform
    )

    tta_loader = DataLoader(
        tta_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    preds_list = []
    for images, _ in tqdm(tta_loader, desc=f"TTA {i+1}/{tta_rounds}"):
        images = images.to(device)
        with torch.no_grad(), torch.amp.autocast("cuda"):
            preds = model(images)
        preds_list.append(preds.softmax(1).cpu().numpy())

    all_preds.append(np.concatenate(preds_list, axis=0))

# 🔹 평균 앙상블 (equal weights)
final_preds = np.mean(np.stack(all_preds), axis=0).argmax(axis=1)

# 🔹 DataFrame 생성
tta_df = pd.DataFrame(tst_dataset.df, columns=["ID", "target"])
tta_df["target"] = final_preds

# 🔹 저장
tta_df.to_csv("pred_5.csv", index=False)
print("✅ Random-Light TTA complete! Saved to pred_5.csv")



🚀 Running Random-Light TTA (5 rounds)



TTA 5/5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.29it/s]

✅ Random-Light TTA complete! Saved to pred_5.csv
